## Setup

### Remove notebook Warning

In [20]:
import warnings
warnings.filterwarnings('ignore')

### Auto-reloard and better matplot

In [21]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Imports

In [22]:
import cv2
import cvzone
from ultralytics import YOLO
import time
from IPython.display import Video
from screeninfo import get_monitors
import os

### Monitor Infos

In [23]:
# Get a list of connected monitors
monitors = get_monitors()

# Get the dimensions of the first monitor
monitor = monitors[0]
monitor_width = monitor.width
monitor_height = monitor.height

## Params

In [24]:
video_name = "P1077418_Balcon_4K25FPS.MP4"

mask_name = "P1077418_Balcon_4K25FPS_MASK.jpg"
apply_mask = True
display_mask = True

frame_by_frame = True


## Main

### Model

In [25]:
facemodel_path = os.path.join(os.path.dirname(os.getcwd()), "Model/yolov8n-face.pt")
if os.path.exists(facemodel_path):
    facemodel = YOLO(facemodel_path)
    pass
else :
    raise FileExistsError


### Loading file

In [26]:
#Path video to analyse and check file exist in ./Videos/video_name
file_video_path = os.path.join(os.path.dirname(os.getcwd()), "Videos", video_name )
if os.path.exists(file_video_path):
    pass
else :
    raise FileExistsError

### Open Video

In [27]:
cap = cv2.VideoCapture(file_video_path)

### Video Infos

In [28]:
fps = cap.get(cv2.CAP_PROP_FPS)
delay = int(1000 / fps)


video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

### Open Mask

In [29]:
if apply_mask :

    mask_path = os.path.join(os.path.dirname(os.getcwd()), "Videos", mask_name )

    if os.path.exists(mask_path):
        mask = cv2.imread(mask_path)

        mask_gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        _, mask_binary = cv2.threshold(mask_gray, 128, 255, cv2.THRESH_BINARY)

    else:
        raise FileExistsError


### Display Video and detection

In [35]:
# Create a window with the size of the video at the calculated position
cv2.namedWindow("Image", cv2.WINDOW_NORMAL)
cv2.moveWindow("Image", (video_width - monitor_width) // 2, (video_height - monitor_height) // 2)
cv2.resizeWindow("Image", monitor_width, monitor_height)

print("debut")
frame_number = 0
timeline = []
while cap.isOpened():



    success, img = cap.read()
    if success:
        frame_number += 1

        if apply_mask:
            img_pretrack = cv2.bitwise_and(img, img, mask=mask_binary)
            if display_mask:
                img = img_pretrack
            else :
                pass

        else :
            img_pretrack= img



        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = facemodel.track(img_pretrack, persist=True)



        # Get the boxes and track IDs
        try:
            boxes = results[0].boxes.xywh
            track_ids = results[0].boxes.id.int().tolist()



        except AttributeError:
            print(f'frame number {frame_number} : no box or no id detected')
            pass


        img = results[0].plot()
        print(f' result : {results[0]}')
        print(f' probs : {results[0].probs}')












        #display FPS
        cv2.putText(img, f'FPS: {int(fps)}', (10  , video_height - 200), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)
        cv2.putText(img, f'Frame : {int(frame_number)}', (10  , video_height - 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)

        cv2.imshow("Image", img)


        if frame_by_frame:
            key = cv2.waitKey(0) & 0xFF
            # if the 'q' key is pressed, exit from loop
            if key == ord("q"):
                cv2.destroyAllWindows()
                break
            #if the 'n' key is pressed, go to next frame
            if key == ord("n"):
                continue
        else :
            #continue loop
            if cv2.waitKey(delay) == ord('q'):
                cv2.destroyAllWindows()
                break


cap.release()
cap = cv2.VideoCapture(file_video_path)


debut

0: 384x640 (no detections), 22.0ms
Speed: 7.0ms preprocess, 22.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
frame number 1 : no box or no id detected
 result : ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'face'}
obb: None
orig_img: array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0]